In [1]:
def FlowMaps(folder,img1,img2):
    import cv2
    import numpy as np
    import LucasKanade
    from LucasKanade import Calc_Flow
    
    #converting to gray_scale
    img1=cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    img2=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    h,w=img1.shape
    flow_map=np.zeros((h,w))
    u,v=Calc_Flow(folder,img1,img2)
    num_corners=2000
    corners = cv2.goodFeaturesToTrack(img1,num_corners,0.01,1)
    x=[]
    y=[]
    num_corners=len(corners)
    for i in range(num_corners):
        x.append(int(corners[i][0][0]))
        y.append(int(corners[i][0][1]))
    im=img1
    for i,j in zip(x,y):
        dx = int(u[j, i] )
        dy = int(v[j, i])
        cv2.arrowedLine(im, (i, j), (i + dx, j + dy), (255,255,255))
        cv2.arrowedLine(flow_map, (i, j), (i + dx, j + dy), (255,255,255))
    return im, flow_map
    

In [2]:
def Mask(folder,im):
    import numpy as np
    import cv2
    im=np.asarray(im)
    #im=im[:,:,0]
    h,w=im.shape
    #print(im.shape,h,w)
    for i in range(0,h,6):
        for j in range(0,w,6):
            if i-1==0 or i-3==0 or j-3==0 or i+1==h or j+3==w or i+3==h or j-1==0 or j+1==w or i+2==h or i-2==0 or j-2==0 or j+2==w:
                pass
            else:
                window=im[i-3:i+3,j-3:j+3]
                if np.count_nonzero(window)>0:
                    im[i-3:i+3,j-3:j+3]=255 
    return im
    

In [3]:
def Segment(folder,img,mask):
    import numpy as np
    img=np.asarray(img)
    mask=np.asarray(mask)
    h,w,_=img.shape
    new_im=np.zeros((h,w))
    for i in range(h):
        for j in range(w):
            if mask[i,j]==0:
                new_im[i,j]=0
            else:
                new_im[i,j]=img[i,j,0]
    return(new_im)


In [6]:
def Box(folder,img1,img):
    import cv2 as cv2
    import numpy as np
 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    image, contours, hierarchy = cv2.findContours(img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    for item in range(len(contours)):
        cnt = contours[item]
        if len(cnt)>1000:
            x,y,w,h = cv2.boundingRect(cnt)
            cv2.rectangle(img1,(x,y),(x+w,y+h),(255,255,0),1)
    return(img1)   


In [7]:
if __name__ == "__main__":
    import numpy as np
    import cv2
    import LucasKanade
    from LucasKanade import Calc_Flow
    import os
    
    image1_name='img10'
    image2_name='img11'
    if not os.path.exists(image1_name): 
        os.makedirs(image1_name) 
    folder='Antique'
    granularity=2
    img1=cv2.imread('./'+folder+'/'+image1_name+'.jpg')
    img2=cv2.imread('./'+folder+'/'+image1_name+'.jpg')
    im,flow_map=FlowMaps(folder,img1,img2)
    cv2.imwrite('./'+image1_name+'/flow_on_im.jpg',im)
    cv2.imwrite('./'+image1_name+'/flow_map.jpg',flow_map)
    mask=Mask(folder,flow_map)
    cv2.imwrite('./'+image1_name+'/mask.jpg',mask)
    seg=Segment(folder,img1,mask)
    cv2.imwrite('./'+image1_name+'/seg.jpg',seg)
    seg_im=cv2.imread('./'+image1_name+'/seg.jpg')
    boxed_im=Box(folder,img1,seg_im)
    cv2.imwrite('./'+image1_name+'/box.jpg',boxed_im)